In [ ]:
"""This section creates the project folders for VRT."""
import pandas as pd
import os

# Read the excel file
df_project = pd.read_excel("D:/ERCOT/AEP Dynamic/MQT/MQT.xlsx",header=None)
project_list = []
for project in df_project.values.tolist():
    num = project[0]
    name = project[1]
    project_list.append(num+' '+name)
    #path = 'D:/ERCOT/AEP Dynamic/MQT/Undone/'+num+' '+name+'/'
    path = 'D:/DMView 3.3.1/CASEs/'+num+' '+name+'/'

    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
        # Create a new directory because it does not exist
        os.makedirs(path)

In [ ]:
"""This section defines functions to check if the project has storage and other."""

def has_ess(project):
    """Input the project (string) e.g. '20INR0213 & 24INR0272 Brushy Creek Solar & Storage SLF'"""

    project = project.lower()
    if ' storage' in project or ' ess' in project or ' bess' in project or ' esc' in project:
        return True
    else:
        return False
    
def has_other(project):
    """Input the project (string) e.g. '20INR0213 & 24INR0272 Brushy Creek Solar & Storage SLF'"""

    project = project.lower()
    if ' solar' in project or ' wind' in project:
        return True
    else:
        return False

In [ ]:
"""This section defines a function to create the ini file given the project number and name."""

def create_ini(project):
    """Input the project (string) e.g. '20INR0213 & 24INR0272 Brushy Creek Solar & Storage SLF'"""

    vrt_path = 'D:/DMView 3.3.1/'
    
    initials = ['lag','lead']

    if has_ess(project):    # Check if there is storage
        scenario = ['_dis','_ch']
    if has_other(project):  # Check if there is solar or wind
        scenario = ['']
    if has_ess(project) and has_other(project):
        scenario = ['','_dis','_ch']
    
    for bess in scenario:
        for initial in initials:
            """Create the ini file."""
            filename = vrt_path + project + '_VRT' + bess + '_' + initial + '.ini'

            """Write the ini file."""
            with open(filename, 'w') as ini:
                ini.write('/' + project + 'for ERCOT dynamic model quality test\n')
                ini.write("/created by Arthur Li - CF Power\n\n")
                ini.write("[Build FS]\nBuild_FS_flag = 1\nPOI_PF = ")
                if initial == 'lag':
                    ini.write('0.95 /lagging \n')
                else:
                    ini.write('-0.95 /leading \n')

                ini.write('\n[Input files]\ninput_path = CASEs\\'+project+'\n')
                ini.write('unconv_casefile ='+project+ bess+'.sav\nmodel_file_lst = [\''+project+'.dyr\']\n')
                ini.write('\n[Tests]\nTest00_FS = [\'FS\', \'20\']\nTest01_LVRT_LEGACY = [\'VOLT\',\'DATAs\\ERCOT_Legacy_LVRT.xlsx\']\n')
                ini.write('Test02_LVRT_VOLT_DIP = [\'VOLT\',\'DATAs\\ERCOT-245_2800_Piecewise_LVRT_(PV-BESS)_10secSpacing_r2.xlsx\']\n')
                ini.write('Test03_HVRT_PREFERED = [\'VOLT\',\'DATAs\\ERCOT-245_2800_HVRT.xlsx\']\n\n[Settings]\nPlot_Flag = 1')

In [ ]:
"""This section defines a function to run the vrt given the project number and name."""

def run_vrt(project):
    """Input the project (string) e.g. '20INR0213 & 24INR0272 Brushy Creek Solar & Storage SLF'"""

    import os
    vrt_path = 'D:/DMView 3.3.1/'
    os.chdir(vrt_path)

    initials = ['lag','lead']

    if has_ess(project):    # Check if there is storage
        scenario = ['_dis','_ch']
    if has_other(project):  # Check if there is solar or wind
        scenario = ['']
    if has_ess(project) and has_other(project):
        scenario = ['','_dis','_ch']

    for bess in scenario:
        for initial in initials:
            filename = vrt_path + project + '_VRT' + bess + '_' + initial + '.ini'
            os.system('python dmview.py "'+filename+'"')
    print('Finish the VRT of '+project+' successfully.\n')


In [ ]:
"""This section defines a function to plot the VRT results and output to a Word file."""

def plot_vrt(project, channels):
    """Input the project (string) e.g. '20INR0213 & 24INR0272 Brushy Creek Solar & Storage SLF'"""
    """Input the channels to be plotted (list) e.g. [4,6,24] """

    import os
    import sys
    import psse35
    import psspy
    import dyntools
    import matplotlib.pyplot as plt
    import glob
    from docx import Document
    from docx.shared import Inches
    from io import BytesIO

    sys_path_PSSE = r'C:\Program Files\PTI\PSSE35\35.5\PSSPY39'  # or where else you find the psspy.pyc
    os_path_PSSE = r'C:\Program Files\PTI\PSSE35\35.5\PSSBIN'  # or where else you find the psse.exe
    sys.path.append(sys_path_PSSE)
    sys.path.append(os_path_PSSE)

    project_path = 'D:/ERCOT/AEP Dynamic/MQT/Undone/'
    os.chdir(project_path+project+'/')

    all_tests = ['TEST00_FS_FS','TEST01_LVRT_LEGACY_VOLT','TEST02_LVRT_VOLT_DIP_VOLT','TEST03_HVRT_PREFERED_VOLT','Test04_HVRT_LEGACY_VOLT']
    tests = all_tests[1:4]  # normal condition
    # tests = all_tests[1:]   # not pass prefered hvrt

    initials = ['lag','lead']

    if has_ess(project):    # Check if there is storage
        scenario = ['_dis','_ch']
    if has_other(project):  # Check if there is solar or wind
        scenario = ['']
    if has_ess(project) and has_other(project):
        scenario = ['','_dis','_ch']

    document = Document()
    document.add_heading('VRT Report for '+project,0)

    for bess in scenario:
        if bess == '_dis':
            subtitle1 = 'Discharging'
        elif bess == '_ch':
            subtitle1 = 'Charging'
        else:
            subtitle1 = ''
        for initial in initials:
            if initial == 'lag':
                subtitle2 = 'Lagging'
            else:
                subtitle2 = 'Leading'

            name = project.upper()+'_VRT' + bess + '_' + initial
            outpath = 'D:\DMView 3.3.1\RESULTs\\'+name
            outlist = []

            document.add_heading(subtitle1 +' '+subtitle2+':')

            for test in tests:
                out = glob.glob(outpath+'\\'+test+'\*.out')[0]
                outlist.append(out)

                outfile = dyntools.CHNF(out)

                short_title, chanid_dict, chandata_dict = outfile.get_data()

                # Plot VRT result
                p_num, q_num, v_num = channels
                time = chandata_dict['time']
                
                title = test+' at '+chanid_dict[p_num][5:]
                labels = ['Real Power','Reactive Power','Voltage']

                fig, (ax1, ax2) = plt.subplots(2, sharex=True)
                fig.suptitle(title)

                ax1.set_ylabel('Real and Reactive Power (pu)')
                ax1.plot(time, chandata_dict[p_num], color='tab:green',)
                ax1.plot(time, chandata_dict[q_num], color='tab:red',)
                ax1.grid(linestyle = '--')

                ax2.set_xlabel('Time (seconds)')
                ax2.set_ylabel('Voltage (pu)')
                ax2.plot(time, chandata_dict[v_num], color='tab:blue',)
                ax2.grid(linestyle = '--')

                fig.tight_layout()  # otherwise the right y-label is slightly clipped
                if test == 'TEST02_LVRT_VOLT_DIP_VOLT':
                    plt.xlim(0,60)
                elif test == 'TEST03_HVRT_PREFERED_VOLT':
                    plt.xlim(0,20)
                else:
                    plt.xlim(0,25)
                fig.legend([ax1, ax2],loc = 'center right', labels=labels)

                memfile = BytesIO()
                plt.savefig(memfile)
                plt.close()
                document.add_picture(memfile, width=Inches(5))
                memfile.close()
                
    document.save(project+'.docx')

In [ ]:
"""This section creates the ini files for VRT."""
import os

project_path = 'D:/ERCOT/AEP Dynamic/MQT/Undone'
project_list = os.listdir(project_path)

for project in project_list:
    create_ini(project)

In [ ]:
"""This section runs the VRT."""
project = '25INR0057 Sombrero Solar'
run_vrt(project)

In [ ]:
"""This section plots the VRT results."""
# project = '20INR0213 & 24INR0272 Brushy Creek Solar & Storage SLF'
plot_vrt(project, channels=[5,9,40])